In [1]:
import torch
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
EPOCHS = 5
BATCH_SIZE = 64

In [3]:
from torchvision import transforms, datasets
#mnist 데이터셋 다운로드, transforms.Compose 텐서로 바꾸면서 0~1 정규화
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('c:/data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,),(0.3081,))
                       ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('c:/data',
                   train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,),
                                            (0.3081,))
                                            ])),
batch_size=BATCH_SIZE, shuffle=True)

100%|██████████| 9912422/9912422 [00:02<00:00, 4036164.51it/s]


Extracting c:/data\MNIST\raw\train-images-idx3-ubyte.gz to c:/data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 5853945.48it/s]

Extracting c:/data\MNIST\raw\train-labels-idx1-ubyte.gz to c:/data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2530114.69it/s]


Extracting c:/data\MNIST\raw\t10k-images-idx3-ubyte.gz to c:/data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting c:/data\MNIST\raw\t10k-labels-idx1-ubyte.gz to c:/data\MNIST\raw



In [4]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.fc3(x) #활성함수 정의 x: linear
        return x

In [5]:
import torch.optim as optim
#과적합과 정규화
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
def train(model, train_loader, optimizer):
    model.train()
    
    for _, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [7]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

[1] Test Loss: 0.5461, Accuracy: 82.77%
[2] Test Loss: 0.4224, Accuracy: 86.61%
[3] Test Loss: 0.3566, Accuracy: 88.68%
[4] Test Loss: 0.3049, Accuracy: 90.60%
[5] Test Loss: 0.2639, Accuracy: 91.92%
